In [ ]:
import math
import sys
import numpy as np

In [ ]:
def divmod_fortran(a, b):
    q = a // b  # Integer division in Python
    r = a % b   # Remainder (modulo operation)
    return q, r

In [ ]:
def split_index(index, nx, ny, nz):
    tmp1, ix = divmod(index - 1, nx)
    tmp2, iy = divmod(tmp1, ny)
    iatom, iz = divmod(tmp2, nz)
    
    ix += 1
    iy += 1
    iz += 1
    iatom += 1

    # Adjust indices for periodic boundary conditions.
    # If an index is greater than the ceiling of (n/2), subtract n to wrap around
    if ix > math.ceil(nx / 2):
        ix = ix - nx
    if iy > math.ceil(ny / 2):
        iy = iy - ny
    if iz > math.ceil(nz / 2):
        iz = iz - nz

    return ix, iy, iz, iatom

In [ ]:
def export_ifc(outputfilename, ifc_phonopy, ss_phonopy):
    # Open the file for writing (it will replace any existing file)
    with open(outputfilename, "w") as f:
        f.write(f" {ss_phonopy:8d} {ss_phonopy:8d}\n")
        for i in range(ss_phonopy):
            for j in range(ss_phonopy):
                f.write(f" {i+1:8d} {j+1:8d}\n")
                for m in range(3):
                    row = "".join(f" {ifc_phonopy[i, j, m, n]:21.15f}" for n in range(3))
                    f.write(row + "\n")

In [ ]:
def parsing_tdep_fcs():
    # if len(sys.argv) < 6:
        # sys.exit("Usage: python tdep2phonopy.py <inputfile> <qx> <qy> <qz> <uc_phonopy>")
    outputfilename = "FORCE_CONSTANTS_2ND_GaN"
    filename = "input_fcs/300K" + "/infile_GaN.forceconstant"
    try:
        qx = 6 # int(sys.argv[2].strip())
        qy = 6 # int(sys.argv[3].strip())
        qz = 4 # int(sys.argv[4].strip())
        uc_phonopy = 4 # int(sys.argv[5].strip())
    except ValueError:
        sys.exit("Error: qx, qy, qz, and uc_phonopy must be integers.")
    
    ss_phonopy = uc_phonopy * qx * qy * qz

    ifc_phonopy = np.zeros((ss_phonopy, ss_phonopy, 3, 3), dtype=np.float64)

    try:
        with open(filename, "r") as f:
            lines = f.readlines()
        print("total length of lines:", len(lines))
    except Exception as e:
        sys.exit(f"Error opening file: {e}")
    
    ptr = 0
    try:
        temp = int(lines[ptr].split()[0])
        print("temp from first line:", temp)
    except Exception as e:
        sys.exit("Error reading the first value from the file.")
    ptr += 1
    if temp != uc_phonopy:
        sys.exit("Error: inconsistent number of atoms in the unitcell!")
    
    ptr += 1
    
    # Allocate neighbor array for each atom in the unitcell.
    neighbor = []
    for i in range(uc_phonopy):
        try:
            neigh = int(lines[ptr].split()[0])
            print("neighbors:", neigh)
        except Exception as e:
            sys.exit("Error reading neighbor count.")
        neighbor.append(neigh)
        ptr += 1
        # Discard next 5*neigh lines.
        ptr += 5 * neigh

    # --- End of first file read block ---
    
    # --- Process pairs of atoms in the supercell ---
    # Loop over all pairs (i, j) where i and j run from 1 to ss_phonopy (inclusive).
    all_ms, fc2_calculated, tdep_displacement_vectors = [], [], []
    for i in range(1, ss_phonopy + 1):
        m_values_for_given_i = []
        for j in range(1, ss_phonopy + 1):
            # Getting correponding supercell indices and atom indices for both ij atoms.
            print("loop over i and j", i, j)
            ix1, iy1, iz1, iatom1 = split_index(i, qx, qy, qz)
            ix2, iy2, iz2, iatom2 = split_index(j, qx, qy, qz)
            
            try:
                with open(filename, "r") as f:
                    file_lines = f.readlines()
                    # print("length of file_lines:", len(file_lines))
            except Exception as e:
                sys.exit(f"Error reopening file: {e}")
            
            ptr_pair = 0
            ptr_pair += 2
            # print("iatom1:", iatom1)

            # If iatom1 is not 1, skip blocks corresponding to atoms before iatom1.
            if iatom1 != 1:
                print("iatom1 is not 1, so skipping blocks")
                for m in range(1, iatom1):
                    # For each previous atom m, skip (5*neighbor(m) + 1) lines.
                    # Note: neighbor list is 0-indexed; Fortran m starts at 1.
                    skip_count = 5 * neighbor[m - 1] + 1
                    ptr_pair += skip_count
            print("ptr_pair:", file_lines[ptr_pair])

            # Read the number of neighbors for atom1 (this value is not used later).
            if ptr_pair < len(file_lines):
                # Read and ignore the number of neighbors.
                ptr_pair += 1
            else:
                continue  # In case of file format issues - as used in fortran script
            
            # Loop over the neighbors of atom1 in TDEP.
            n_neighbors = neighbor[iatom1 - 1]
            print(f"{n_neighbors} neighbors for {i} - {j} pair almaBTE atom; whose iatoms are {iatom1} & {iatom2}")
            
            found = False
            for m in range(n_neighbors):
                if ptr_pair >= len(file_lines):
                    break
                try:
                    atomtdep = int(file_lines[ptr_pair].split()[0])
                    print(f"neighbor index of TDEP atom is {atomtdep} from the line: {file_lines[ptr_pair]}")
                except Exception as e:
                    sys.exit("Error reading atomtdep.")
                ptr_pair += 1
                
                if atomtdep == iatom2:
                    # Read the cell displacement information using lattice vectors
                    try:
                        celltdep = list(map(float, file_lines[ptr_pair].split()))
                    except Exception as e:
                        sys.exit("Error reading celltdep.") 
                    ptr_pair += 1
                    # Check if the relative indices match.
                    if celltdep == [ix2 - ix1, iy2 - iy1, iz2 - iz1]:
                        tdep_displacement_vectors.append(celltdep)
                        # Read three lines of 3 floats each and assign them to ifc_phonopy array.
                        for k in range(3):
                            try:
                                numbers = list(map(float, file_lines[ptr_pair].split()))
                            except Exception as e:
                                sys.exit("Error reading force constants.")
                            ptr_pair += 1
                            # Assign the three values.
                            for n in range(3):
                                ifc_phonopy[i - 1, j - 1, k, n] = numbers[n]
                            fc2_calculated.append(numbers)
                        found = True
                        m_values_for_given_i.append(m+1)
                        print(f"Found the {m+1}th (neighbor) TDEP atom match to {j}th pair atom of {i}th almaBTE atom")
                        break  # Exit the neighbor loop.
                    else:
                        # Discard the next 3 lines which includes the block
                        ptr_pair += 3
                else:
                    # Discard the next 4 lines which includes the block and lattice displacement vector
                    ptr_pair += 4

            # End of neighbor loop
            # (If no matching neighbor was found, ifc_phonopy remains zero for this (i,j) pair)
        print(len(m_values_for_given_i))
        all_ms.append(m_values_for_given_i)
    print(all_ms)
    np.save("all_ms.npy", all_ms)
    # Export the resulting ifc_phonopy array.
    export_ifc(outputfilename, ifc_phonopy, ss_phonopy)
    print("Export complete. Check the file 'FORCE_CONSTANTS_2ND'.")
    return all_ms, fc2_calculated, tdep_displacement_vectors

In [ ]:
if __name__ == "__main__":
    all_ms, fc2_calculated, tdep_displacement_vectors = parsing_tdep_fcs()